In [4]:
import json
import numpy as np
import csv

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import nltk

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm


from sklearn import preprocessing

pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [ ]:
# import relevant LMs package

import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score
from transformers import GPT2Model, GPT2Tokenizer

In [ ]:
# cheak to use gpu or cpu
if torch.cuda.is_available():
    print("Use gpu")
    device = torch.device("cuda")
else:
    print("Use cpu")
    device = torch.device("cpu")

In [ ]:
device = "cuda:0"
torch.cuda.empty_cache()
# torch.cuda.reset_max_memory_allocated()

In [ ]:
path='C:\\Users\\wz\\Desktop\\DBpedia_query\\train\\DBpedia_query_train.csv'
DBpedia = pd.read_csv(path)
DBpedia

In [ ]:
# next token prediction
from transformers import AutoModelForCausalLM, AutoTokenizer


print("\nBegin next-word using HF GPT-2 demo ")

toker = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")


In [ ]:
from  transformers import GPT2TokenizerFast, GPT2LMHeadModel

base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
vocabulary = base_tokenizer.get_vocab()
vocabulary['University']

text = 'University of Wisconsin%E2%80%93Madison'
sub = 'University'
text_ids = base_tokenizer.encode(sub, return_tensors = 'pt')
text_ids.item()

In [ ]:
# import torch

# from  transformers import GPT2TokenizerFast, GPT2LMHeadModel
# from torch.nn import CrossEntropyLoss

# model = GPT2LMHeadModel.from_pretrained("gpt2")
# model.eval()
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# ### We calculate the hidden_states and the past of the common left part of the sentence
# past = "I like sitting in my new chair and"
# past_tokenize_input = tokenizer.tokenize(past)
# past_tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(past_tokenize_input)])

# past_last_hidden_state, past = model.transformer(past_tensor_input)

# print(past_last_hidden_state)

# def score(sentence, past, past_last_hidden_state, past_tensor_input):
#     tokenize_input = tokenizer.tokenize(sentence, )
#     tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])

#     ###the following code is slightly modified from https://github.com/huggingface/transformers/blob/09a2f40684f77e62d0fd8485fe9d2d610390453f/src/transformers/modeling_gpt2.py#L604
#     ###now we calculate the right part of the sentence with the already calculated past
#     transformer_outputs = model.transformer(
#             tensor_input,
#             attention_mask=None,
#             token_type_ids=None,
#             position_ids=None,
#             head_mask=None,
#             inputs_embeds=None,
#             use_cache=None,
#             output_attentions=None,
#             output_hidden_states=None,
#         )
    
#     # print(past_last_hidden_state)
        
#     ###and concatenate the output of with the hidden_state of the left part of the sentence
#     hidden_states = torch.cat((past_last_hidden_state, transformer_outputs[0]), dim=1)
    
#     ###the following part is exactly the same as https://github.com/huggingface/transformers/blob/09a2f40684f77e62d0fd8485fe9d2d610390453f/src/transformers/modeling_gpt2.py#L604
#     lm_logits = model.lm_head(hidden_states)

#     labels_input = torch.cat((past_tensor_input, tensor_input), dim=1)

#     # Shift so that tokens < n predict n
#     shift_logits = lm_logits[..., :-1, :].contiguous()
#     shift_labels = labels_input[..., 1:].contiguous()
#     # Flatten the tokens
#     loss_fct = CrossEntropyLoss()
#     loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
#     return -loss.item()

# candidates = ["watch", "run", "think", "apple", "light"]

# sent_template = " {} about life"

# print({candidate: score((sent_template.format(candidate)), past, past_last_hidden_state, past_tensor_input) for candidate in candidates})

In [ ]:
# get a subgraph that include the true answer with input row
def get_label(row, n):
    text = DBpedia.loc[3,"query"]
    true_answer = DBpedia.loc[3,"response"]
    DBpedia_s = DBpedia.sample(n-1)
    DBpedia_row = DBpedia.iloc[[row]]
    DBpedia_sub = pd.concat([DBpedia_s,DBpedia_row], axis=0)
    # get labels
    le = preprocessing.LabelEncoder()
    all_response = DBpedia_sub.loc[:,'response'].values
    le.fit(all_response)
    label_value = le.transform(all_response)
    # form a dict labels
    labels = dict()
    for i in range(len(all_response)):
        labels[all_response[i]] = label_value[i]
    # print(len(all_response))
    return text, true_answer, all_response, label_value, labels

In [ ]:
# all_response = all_response[0:100]

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def next_word_prediction(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss = model(tensor_input, labels=tensor_input)
    return -loss[0].item()

def outcome(candidates, sentence, true_answer):
    # candidates = all_response
    sent_template = sentence + " {}"
    # print({candidate: next_word_prediction(sent_template.format(candidate)) for candidate in candidates})
    response_prob = {candidate: next_word_prediction(sent_template.format(candidate)) for candidate in candidates}
    max_key = max(response_prob, key=lambda k: response_prob[k])
    # loss from true response 
    loss = response_prob[max_key] - response_prob[true_answer]
    return max_key, loss

In [ ]:
def next_word_GPT2(row, n):
    # print( get_label(3,100))
    text, true_answer, all_response, label_value, labels = get_label(3,100)
    max_key, loss = outcome(all_response,text,true_answer)
    return max_key, loss

In [ ]:
next_word_GPT2(10,100)

In [ ]:
# # get a bound for loss for 3 trail
# for it in range(3):
#     array = []
#     for i in tqdm(range(len(DBpedia))):
#         array.append(next_word_GPT2(i,100))
#     print("The median is ", np.median(array))

In [ ]:
# np.median(array)

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt
tqdm.pandas()

from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt

config = {
    "lm_name": "lvwerra/gpt2",
    "ref_lm_name": "lvwerra/gpt2",
    "tk_name": "gpt2",
    "steps": 6400,
    "batch_size": 64,
    "forward_batch_size": 8,
    "ppo_epochs": 4,   
    "txt_in_len": 5,
    "txt_out_len": 15,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "seed": 1,
}

np.random.seed(config['seed'])

In [ ]:
# trails
wandb.init(name='trails #7', project='gpt2-query', config=config)

In [ ]:
# load models
gpt2_model = GPT2HeadWithValueModel.from_pretrained("gpt2")
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(config['tk_name'])

In [ ]:
_ = gpt2_model.to(device)
_ = gpt2_model_ref.to(device)

In [ ]:
wandb.watch(gpt2_model, log='all')

In [ ]:
# bert classifier
# unused
device = "cuda:0"
sentiment_model = AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb")
sentiment_tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")


In [ ]:
# rename
df = DBpedia.copy()
ctrl_str = ['[false]', '[true]']
ctrl_tokens = dict((s, gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [ ]:
# special token
df['tokens'] = df['query'].apply(lambda x: gpt2_tokenizer.encode(' '+str(x), return_tensors="pt").to(device))
df['tokens'] = df['tokens'].apply(lambda x: x[0])
df['tokens'] = df['tokens'].apply(lambda x: x[:2])

# dropping invalid length in token column
drop_row_index = []
for i in range(1000):

    if len(df['tokens'].loc[i]) < 2:
        #df['tokens'].loc[i] = df['tokens'].loc[0].cuda().detach().cpu().clone().numpy()
        drop_row_index.append(i)
print(drop_row_index)
df = df.drop(df.index[drop_row_index])

### load trained model from finetune on facts 0/1

In [ ]:
class SimpleGPT2SequenceClassifier(nn.Module):
    def __init__(self, hidden_size: int, num_classes:int ,max_seq_len:int, gpt_model_name:str):
        super(SimpleGPT2SequenceClassifier,self).__init__()
        self.gpt2model = GPT2Model.from_pretrained(gpt_model_name)
        self.fc1 = nn.Linear(hidden_size*max_seq_len, num_classes)

        
    def forward(self, input_id, mask):
        """
        Args:
                input_id: encoded inputs ids of sent.
        """
        gpt_out, _ = self.gpt2model(input_ids=input_id, attention_mask=mask, return_dict=False)
        batch_size = gpt_out.shape[0]
        linear_output = self.fc1(gpt_out.view(batch_size,-1))
        return linear_output

In [ ]:
model_new = SimpleGPT2SequenceClassifier(hidden_size=768, num_classes=2, max_seq_len=10, gpt_model_name="gpt2")
model_new.load_state_dict(torch.load("C:\\Users\\wz\\Desktop\\Finetune on 0,1\\model\\gpt2-text-classifier-model.pt"))
model_new.eval()

### use the model we trained on fact 

In [ ]:
example_text = """
Fairview, California country Iran
"""
def fact_eval(text):
    fixed_text = " ".join(example_text.lower().split())
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token

    model_input = tokenizer(fixed_text, padding='max_length', max_length=10, truncation=True, return_tensors="pt")
    mask = model_input['attention_mask'].cpu()
    input_id = model_input["input_ids"].squeeze(1).cpu()

    output = model_new(input_id, mask)
    prob = torch.nn.functional.softmax(output, dim=1)[0]
    pred_fact = output.argmax(dim=1).item()
    # 0 for false, 1 for true
    return pred_fact

In [ ]:
# fact_eval(example_text)

In [ ]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i]=='[false]':
            logit[i] = -logit[i]
        elif task[i]=='[true]':
            pass
        else:
            raise ValueError('task has to be in [0, 1, 2]!')
    return logit

In [ ]:
### KG check to modify reward value 

# given a range i.e. i*fbs to (i+1)*fbs
# should output a tensor([ 0, 0,  1, 1], device='cuda:0')

# pred is the reponse in batch_response
# query is the query_list

def KG_check(lower, upper):
    factuals = []
    for i in range(lower,upper):
        query = DBpedia.loc[i,"query"]
        response = DBpedia.loc[i,"response"]
        text = "" + query + response
        # print(text)
        factual = fact_eval(text)
        factuals.append(factual)
    # add variant 
    tensor_array = [i + random.random() for i in factuals ]
    return torch.cuda.LongTensor(tensor_array)


In [ ]:
KG_check(3, 7)

In [ ]:
# batch_size: 32
# forward_batch_size: 4 

ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset and annotate tasks
    df_batch = df.sample(config['batch_size'])
    task_list = choices(ctrl_str, k=config['batch_size'])
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])
    query_list = df_batch['query'].tolist()
    game_data['query'] = [t+q for t,q in zip(task_list, query_list)]
    
    
    #print(task_tensors)
    query_tensors = torch.stack((df_batch['tokens'].tolist()))
    #print("before ",query_tensors)
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    
    #### get response from gpt2
    t = time.time()
    total_length = config['txt_in_len']+config['txt_out_len']
    response_tensors_array = []
    for i in range(int(config['batch_size']/fbs)):
        response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],
                                     txt_len=config['txt_out_len'])
        response_tensors_array.append(response)
    response_tensors = torch.cat(response_tensors_array)
#         #print(int(config['batch_size']/fbs))
#         # 8 loops in total
#         # 0-4, 5-8, 9-12...
        
#         # this is troublesome 
#         # error: probability tensor contains either `inf`, `nan` or element < 0
#         #  next_token = torch.multinomial(probs, num_samples=1).squeeze(1) in gpt2.py function respond_to_batch
        
#         # response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],txt_len=config['txt_out_len'])
#         # we want a response of size [i*fbs:(i+1)*fbs]*1
#         response_tensors = []
#         for i in range(i*fbs, (i+1)*fbs):
            
#             seq = query_list[i]
#             inpts = toker(seq, return_tensors="pt")
#             inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
            
#             key, loss = next_word_GPT2(i,100)
            
#             pred_id = base_tokenizer.encode(key, return_tensors = 'pt')
#             # now it force response list into one tensor
#             response = (pred_id[0])[1] # todo change it  should be one id not a list
#             # print(response)
#             response_tensors.append(response.item())
#         response_tensors = torch.cuda.LongTensor(response_tensors)
#         response_tensors_array.append(response_tensors)
   
#     # response_tensors.size = 32 * 1  
#     response_tensors = torch.cat(response_tensors_array,axis = -1 )
#     response_tensors  = torch.reshape(response_tensors, (-1, 1)) # force size of response_tensors.size = 32 * 1  rather that 32 length list
#     # print(response_tensors)
#     # print([gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])])
#     # print(response_tensors)
    batch_response = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])] # list of length 32
    game_data['response'] = batch_response
    timing['time/get_response'] = time.time()-t
    #print( game_data['response'])

    #### tokenize text for sentiment analysis
    t = time.time()
    texts = [q + r for q,r in zip(query_list, game_data['response'])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    timing['time/build_input_sentiment_unused'] = time.time()-t
        
    #### get KG score
    t = time.time()
    pos_logits = []
    for i in range(int(config['batch_size']/fbs)):
        device = torch.device("cuda")
        # modify the rewards based on the KG
        res = KG_check(i*fbs, (i+1)*fbs)
        pos_logits.append(res)
    
    rewards = pos_logit_to_reward(torch.cat(pos_logits), task_list)
    timing['time/get_factual_preds'] = time.time()-t


    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    # enforce nan to zero for wandb to run
    stats = np.nan_to_num(stats)
    # print(stats)
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    # comment to remove the nan issues
    logs.update(stats)
    
    # only support for float tensor
    rewards = rewards.type(torch.FloatTensor)
    
    logs['env/reward_mean'] = torch.mean((rewards)).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    for ctrl_s in ctrl_str:
        key = 'env/reward_'+ctrl_s.strip('[]')
        logs[key] = np.mean([r for r, t in zip(logs['env/reward_dist'], task_list) if t==ctrl_s])
#     WandbCallback(log=None)
    wandb.log(logs)

In [47]:
# fact_eval(example_text)

0

In [41]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i]=='[false]':
            logit[i] = -logit[i]
        elif task[i]=='[true]':
            pass
        else:
            raise ValueError('task has to be in [0, 1, 2]!')
    return logit

In [50]:
### KG check to modify reward value 

# given a range i.e. i*fbs to (i+1)*fbs
# should output a tensor([ 0, 0,  1, 1], device='cuda:0')

# pred is the reponse in batch_response
# query is the query_list

def KG_check(lower, upper):
    factuals = []
    for i in range(lower,upper):
        query = DBpedia.loc[i,"query"]
        response = DBpedia.loc[i,"response"]
        text = "" + query + response
        # print(text)
        factual = fact_eval(text)
        factuals.append(factual)
    # add variant 
    tensor_array = [i + random.random() for i in factuals ]
    return torch.cuda.LongTensor(tensor_array)


In [51]:
KG_check(3, 7)

C:\Users\wz\AppData\Local\Temp\ipykernel_12672\4003785611.py:20: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array)


tensor([0, 0, 0, 0], device='cuda:0')

In [54]:
# batch_size: 32
# forward_batch_size: 4 

ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset and annotate tasks
    df_batch = df.sample(config['batch_size'])
    task_list = choices(ctrl_str, k=config['batch_size'])
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])
    query_list = df_batch['query'].tolist()
    game_data['query'] = [t+q for t,q in zip(task_list, query_list)]
    
    
    #print(task_tensors)
    query_tensors = torch.stack((df_batch['tokens'].tolist()))
    #print("before ",query_tensors)
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    
    #### get response from gpt2
    t = time.time()
    total_length = config['txt_in_len']+config['txt_out_len']
    response_tensors_array = []
    for i in range(int(config['batch_size']/fbs)):
        response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],
                                     txt_len=config['txt_out_len'])
        response_tensors_array.append(response)
    response_tensors = torch.cat(response_tensors_array)
#         #print(int(config['batch_size']/fbs))
#         # 8 loops in total
#         # 0-4, 5-8, 9-12...
        
#         # this is troublesome 
#         # error: probability tensor contains either `inf`, `nan` or element < 0
#         #  next_token = torch.multinomial(probs, num_samples=1).squeeze(1) in gpt2.py function respond_to_batch
        
#         # response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],txt_len=config['txt_out_len'])
#         # we want a response of size [i*fbs:(i+1)*fbs]*1
#         response_tensors = []
#         for i in range(i*fbs, (i+1)*fbs):
            
#             seq = query_list[i]
#             inpts = toker(seq, return_tensors="pt")
#             inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
            
#             key, loss = next_word_GPT2(i,100)
            
#             pred_id = base_tokenizer.encode(key, return_tensors = 'pt')
#             # now it force response list into one tensor
#             response = (pred_id[0])[1] # todo change it  should be one id not a list
#             # print(response)
#             response_tensors.append(response.item())
#         response_tensors = torch.cuda.LongTensor(response_tensors)
#         response_tensors_array.append(response_tensors)
   
#     # response_tensors.size = 32 * 1  
#     response_tensors = torch.cat(response_tensors_array,axis = -1 )
#     response_tensors  = torch.reshape(response_tensors, (-1, 1)) # force size of response_tensors.size = 32 * 1  rather that 32 length list
#     # print(response_tensors)
#     # print([gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])])
#     # print(response_tensors)
    batch_response = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])] # list of length 32
    game_data['response'] = batch_response
    timing['time/get_response'] = time.time()-t
    #print( game_data['response'])

    #### tokenize text for sentiment analysis
    t = time.time()
    texts = [q + r for q,r in zip(query_list, game_data['response'])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    timing['time/build_input_sentiment_unused'] = time.time()-t
        
    #### get KG score
    t = time.time()
    pos_logits = []
    for i in range(int(config['batch_size']/fbs)):
        device = torch.device("cuda")
        # modify the rewards based on the KG
        res = KG_check(i*fbs, (i+1)*fbs)
        pos_logits.append(res)
    
    rewards = pos_logit_to_reward(torch.cat(pos_logits), task_list)
    timing['time/get_factual_preds'] = time.time()-t


    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    # enforce nan to zero for wandb to run
    stats = np.nan_to_num(stats)
    # print(stats)
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    # comment to remove the nan issues
    logs.update(stats)
    
    # only support for float tensor
    rewards = rewards.type(torch.FloatTensor)
    
    logs['env/reward_mean'] = torch.mean((rewards)).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    for ctrl_s in ctrl_str:
        key = 'env/reward_'+ctrl_s.strip('[]')
        logs[key] = np.mean([r for r, t in zip(logs['env/reward_dist'], task_list) if t==ctrl_s])
#     WandbCallback(log=None)
    wandb.log(logs)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\wz\AppData\Local\Temp\ipykernel_12672\4003785611.py:20: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array)
100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [4:29:05<00:00, 161.46s/it]
